In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import joblib

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import lightgbm as lgb
import itertools
import ast
import time

from sklearn.pipeline import make_pipeline
from sklearn import model_selection

from sklearn.preprocessing import RobustScaler, LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, KFold
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.impute import SimpleImputer

import warnings

warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')

OSError: dlopen(/Users/nikk/git/venv/lib/python3.7/site-packages/lightgbm/lib_lightgbm.so, 6): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib
  Referenced from: /Users/nikk/git/venv/lib/python3.7/site-packages/lightgbm/lib_lightgbm.so
  Reason: image not found

In [8]:
train = pd.read_csv('DATA/train.csv')
test = pd.read_csv('DATA/test.csv')
sample_submission = pd.read_csv('DATA/sample_submission.csv')

In [9]:
X=train.drop(["revenue"], axis=1)
y = np.log1p(train['revenue'])



In [10]:
def lab_encode(df):
    for col in ['production_countries', 'all_genres', 'original_language']:
        df[col]= LabelEncoder().fit_transform(df[col])


pipeline = Pipeline([
    #("lab_encode" , lab_encode()),
    ("Rbt_scaler", RobustScaler())
])
prepared = pipeline.fit_transform(X)
pretest = pipeline.transform(test)



NameError: name 'Pipeline' is not defined

In [6]:
params = {'num_leaves': 30,
         'min_data_in_leaf': 20,
         'objective': 'regression',
         'max_depth': 5,
         'learning_rate': 0.01,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9,
         "bagging_seed": 11,
         "metric": 'rmse',
         "lambda_l1": 0.2,
         "verbosity": -1}

lgb_model = lgb.LGBMRegressor(**params, n_estimators = 10000, nthread = 4, n_jobs = -1)


NameError: name 'lgb' is not defined

In [11]:
Random_Search_Params ={
    "max_depth": [4],
    "min_data_in_leaf": [15],
    'learning_rate': [0.01],
    'num_leaves': [40],
    'boosting_type' : ['gbdt'],
    'objective' : ['regression'],
    'random_state' : [501], 
    }

n_HP_points_to_test = 50

random_search = RandomizedSearchCV(
    estimator=lgb_model, param_distributions= Random_Search_Params, 
    n_iter=n_HP_points_to_test,
    cv=3,
    refit=True,
    random_state=314,
    verbose=True)

random_search.fit(X, y)

# Using parameters already set above, replace in the best from the random search
params['learning_rate'] = random_search.best_params_['learning_rate']
params['max_depth'] = random_search.best_params_['max_depth']
params['num_leaves'] = random_search.best_params_['num_leaves']


NameError: name 'RandomizedSearchCV' is not defined

In [12]:
joblib.dump(random_search.best_estimator_, "../model/model.joblib", compress = 1)
#final_model = random_search.best_estimator_
#final_predict = final_model.predict(test)
#len(final_predict)

NameError: name 'random_search' is not defined

In [15]:
# sample_submission['revenue'] = np.expm1(final_prediction + y_pred)/2
#sample_submission['revenue'] = np.expm1(final_predict)
#sample_submission.to_csv('final_sub.csv', index=False)